In [30]:
import requests
from urllib.parse import urlparse, urljoin
from bs4 import BeautifulSoup
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import os

In [31]:
url = "https://csirt.bppt.go.id"
soup = BeautifulSoup(requests.get(url).content, "html.parser")
data1 = soup.get_text(strip=True)
data2 = soup.get_text()

In [32]:
with open ("kemarin.txt", "w") as f:
    f.write(data1)

In [33]:
with open ("sekarang.txt", "w") as f:
    f.write(data1)

In [34]:
data_files = [doc for doc in os.listdir() if doc.endswith('.txt')]
data_notes =[open(File).read() for File in  data_files]
vectorize = lambda Text: TfidfVectorizer().fit_transform(Text).toarray()
similarity = lambda doc1, doc2: cosine_similarity([doc1, doc2])

In [35]:
vectors = vectorize(data_notes)
s_vectors = list(zip(data_files, vectors))

In [36]:
def check_similarity():
    similarity_results = set()
    global s_vectors
    for data_a, text_vector_a in s_vectors:
        new_vectors =s_vectors.copy()
        current_index = new_vectors.index((data_a, text_vector_a))
        del new_vectors[current_index]
        for data_b , text_vector_b in new_vectors:
            sim_score = similarity(text_vector_a, text_vector_b)[0][1]
            data_pair = sorted((data_a, data_b))
            score = (data_pair[0], data_pair[1],sim_score)
            similarity_results.add(score)
    return similarity_results


In [37]:
for data in check_similarity():
    print(data)

('base.txt', 'kemarin.txt', 1.0000000000000004)
('kemarin.txt', 'sekarang.txt', 1.0000000000000004)
('base.txt', 'sekarang.txt', 1.0000000000000004)


In [38]:
def check_kemarin_sekarang():
    data_eksisting = check_similarity()
    if list(data_eksisting)[0][2] < 0.85:
        return("false")